In [ ]:
import os
import os.path
import matplotlib.pyplot as plt

import numpy as np
import tensorflow as tf

import input_data
import cats_vs_dogs_cnn
import tools

In [ ]:
BATCH_SIZE = 32
N_CLASSES = 2
CAPACITY = 2000
IMG_W = 208
IMG_H = 208
learning_rate = 0.001
MAX_STEP = 15000
IS_PRETRAIN =True

def train():
    train_dir = "./data/train/"
    train_log_dir = './save_model/'
    #test_dir = './data/test1/'
    
    train_image_list, train_label_list = input_data.get_files(train_dir)
    train_image_batch, train_label_batch = input_data.get_batch(train_image_list, train_label_list, IMG_W, IMG_H, BATCH_SIZE, CAPACITY)

    logits = cats_vs_dogs_cnn.cats_vs_dogs_cnn(train_image_batch, N_CLASSES, IS_PRETRAIN)
    loss = tools.loss(logits, train_label_batch)
    accuracy = tools.accuracy(logits, train_label_batch)
    my_global_step = tf.Variable(0, name='global_step', trainable=False)
    train_op = tools.optimize(loss, learning_rate, my_global_step)
    
    x = tf.placeholder(tf.float32, shape=[BATCH_SIZE, IMG_W, IMG_H, 3])
    y_ = tf.placeholder(tf.int16, shape=[BATCH_SIZE, N_CLASSES])
    
    saver = tf.train.Saver(tf.global_variables())
    
    init = tf.global_variables_initializer()
    with tf.Session() as sess:
        sess.run(init)

        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(sess=sess, coord=coord)
        try:
            for step in np.arange(MAX_STEP):
                if coord.should_stop():
                    break
                tra_images, tra_labels = sess.run([train_image_batch, train_label_batch])
                _, tra_loss, tra_acc = sess.run([train_op, loss, accuracy], feed_dict={x:tra_images, y_:tra_labels})

                if step%50 == 0 or (step+1) == MAX_STEP:
                    print('step: {}, loss: {:.4f}, accuracy: {:.4f}%%'.format(step, tra_loss, tra_acc)) 
#                     summary_str = sess.run(summary_op)
#                     tra_summary_writer.add_summary(summary_str, step)
                if step%2000 == 0 or (step+1) == MAX_STEP:
                    checkpoint_path = os.path.join(train_log_dir, 'model.ckpt')
                    saver.save(sess, checkpoint_path, global_step=step)

        except tf.errors.OutOfRangeError:
            print('Done training -- epoch limit reached')
        finally:
            coord.request_stop()
        coord.join(threads)

In [ ]:
if __name__ == '__main__':
    train()